# original NNM

In [ ]:
import argparse
import torch
import yaml
from termcolor import colored
from utils.common_config import get_val_dataset, get_val_transformations, get_val_dataloader,\
                                get_model
from utils.evaluate_utils import hungarian_evaluate
from utils.memory import MemoryBank 
from utils.utils import fill_memory_bank
from PIL import Image

from data.custom_dataset import NeighborsDataset
from utils.common_config import get_feature_dimensions_backbone
import torch.nn.functional as F

In [ ]:
@torch.no_grad()
def get_prototypes(config, predictions, features, model, topk=3):
    import torch.nn.functional as F

    # Get topk most certain indices and pred labels
    print('Get topk')
    probs = predictions['probabilities']
    n_classes = probs.shape[1]
    dims = features.shape[1]
    max_probs, pred_labels = torch.max(probs, dim = 1)
    indices = torch.zeros((n_classes, topk))
    for pred_id in range(n_classes):
        probs_copy = max_probs.clone()
        mask_out = ~(pred_labels == pred_id)
        probs_copy[mask_out] = -1
        conf_vals, conf_idx = torch.topk(probs_copy, k = topk, largest = True, sorted = True)
        indices[pred_id, :] = conf_idx

    # # Get corresponding features
    # selected_features = torch.index_select(features, dim=0, index=indices.view(-1).long())
    # selected_features = selected_features.unsqueeze(1).view(n_classes, -1, dims)

    # # Get mean feature per class
    # mean_features = torch.mean(selected_features, dim=1)

    # # Get min distance wrt to mean
    # diff_features = selected_features - mean_features.unsqueeze(1)
    # diff_norm = torch.norm(diff_features, 2, dim=2)

    # # Get final indices
    # _, best_indices = torch.min(diff_norm, dim=1)
    # one_hot = F.one_hot(best_indices.long(), indices.size(1)).byte()
    # proto_indices = torch.masked_select(indices.view(-1), one_hot.view(-1))
    # proto_indices = proto_indices.int().tolist()
    # return proto_indices
    return indices

In [ ]:
def visualize_indices(indices, dataset, hungarian_match, config):
    import matplotlib.pyplot as plt
    import numpy as np

    # for idx in indices:
    #     img = np.array(dataset.get_image(idx)).astype(np.uint8)
    #     img = Image.fromarray(img)
    #     plt.figure()
    #     plt.axis('off')
    #     plt.imshow(img)
    #     #plt.show()
    #     plt.savefig(config['train_db_name']+'_'+str(idx)+'_.png', dpi=500, bbox_inches='tight')

    for c in range(indices.shape[0]) :
        indices_c = indices[c,:].int().tolist()
        for idx in indices_c:
            img = np.array(dataset.get_image(idx)).astype(np.uint8)
            img = Image.fromarray(img)
            plt.figure()
            plt.axis('off')
            plt.imshow(img)
            #plt.show()
            plt.savefig('./'+config['train_db_name']+'/'+config['train_db_name']+'_'+str(c)+'_'+str(idx)+'.png', dpi=500, bbox_inches='tight')

In [ ]:
@torch.no_grad()
def get_predictions(p, dataloader, model, return_features=False):
    # Make predictions on a dataset with neighbors
    model.eval()
    predictions = [[] for _ in range(p['num_heads'])]
    probs = [[] for _ in range(p['num_heads'])]
    targets = []
    if return_features:
        ft_dim = get_feature_dimensions_backbone(p)
        features = torch.zeros((len(dataloader.sampler), ft_dim)).cuda()
    
    if isinstance(dataloader.dataset, NeighborsDataset): # Also return the neighbors
        key_ = 'anchor'
        include_neighbors = True
        neighbors = []

    else:
        key_ = 'image'
        include_neighbors = False

    ptr = 0
    for batch in dataloader:
        images = batch[key_].cuda(non_blocking=True)
        bs = images.shape[0]
        res = model(images, forward_pass='return_all')
        output = res['output']
        if return_features:
            features[ptr: ptr+bs] = res['features']
            ptr += bs
        for i, output_i in enumerate(output):
            predictions[i].append(torch.argmax(output_i, dim=1))
            probs[i].append(F.softmax(output_i, dim=1))
        targets.append(batch['target'])
        if include_neighbors:
            neighbors.append(batch['possible_neighbors'])

    predictions = [torch.cat(pred_, dim = 0).cpu() for pred_ in predictions]
    probs = [torch.cat(prob_, dim=0).cpu() for prob_ in probs]
    targets = torch.cat(targets, dim=0)

    if include_neighbors:
        neighbors = torch.cat(neighbors, dim=0)
        out = [{'predictions': pred_, 'probabilities': prob_, 'targets': targets, 'neighbors': neighbors} for pred_, prob_ in zip(predictions, probs)]

    else:
        out = [{'predictions': pred_, 'probabilities': prob_, 'targets': targets} for pred_, prob_ in zip(predictions, probs)]

    if return_features:
        return out, features
    else:
        return out

In [ ]:
FLAGS = argparse.ArgumentParser(description='Evaluate models from the model zoo')
FLAGS.add_argument('--config_exp', help='Location of config file', default='configs/scan/scan_cifar20.yml')
FLAGS.add_argument('--model', help='Location where model is saved', default="cifar20.pth.tar")
FLAGS.add_argument('--visualize_prototypes', action='store_true', 
                    help='Show the prototpye for each cluster', default=False)
args = FLAGS.parse_args([])

In [ ]:
# Read config file
print(colored('Read config file {} ...'.format(args.config_exp), 'blue'))
with open(args.config_exp, 'r') as stream:
    config = yaml.safe_load(stream)
config['batch_size'] = 256 # To make sure we can evaluate on a single 1080ti
print(config)

In [ ]:
# Get dataset
print(colored('Get validation dataset ...', 'blue'))
transforms = get_val_transformations(config)
dataset = get_val_dataset(config, transforms)
dataloader = get_val_dataloader(config, dataset)
print('Number of samples: {}'.format(len(dataset)))

# Get model
print(colored('Get model ...', 'blue'))
model = get_model(config)
print(model)

In [ ]:
# Read model weights
print(colored('Load model weights ...', 'blue'))
state_dict = torch.load(args.model, map_location='cpu')

if config['setup'] in ['simclr', 'moco', 'selflabel']:
    model.load_state_dict(state_dict)

elif config['setup'] == 'scan':
    model.load_state_dict(state_dict['model'])

else:
    raise NotImplementedError

In [ ]:
model.cuda()

In [ ]:
dataset.classes

In [ ]:
if config['setup'] in ['scan', 'selflabel']:
    print(colored('Perform evaluation of the clustering model (setup={}).'.format(config['setup']), 'blue'))
    head = state_dict['head'] if config['setup'] == 'scan' else 0
    predictions, features = get_predictions(config, dataloader, model, return_features=True)
    clustering_stats = hungarian_evaluate(head, predictions, dataset.classes, 
                                            compute_confusion_matrix=True, 
                        confusion_matrix_file=None)
    print(clustering_stats)
    if args.visualize_prototypes:
        prototype_indices = get_prototypes(config, predictions[head], features, model)
        visualize_indices(prototype_indices, dataset, clustering_stats['hungarian_match'], config)
else:
    raise NotImplementedError

## train GAN

In [ ]:
import importlib

In [ ]:
import gan_attack
importlib.reload(gan_attack)

In [ ]:
use_cuda=True
image_nc=3
epochs = 60
batch_size = 200
BOX_MIN = 0
BOX_MAX = 1
model_num_labels = 20
device = 'cuda'

In [ ]:
GAN = gan_attack.GAN_Attack(device, model, model_num_labels, image_nc, BOX_MIN, BOX_MAX, 'cifar100')

In [ ]:
import sys

# Holding the original output object. i.e. console out
orig_stdout = sys.stdout

# Opening the file to write file deletion logs.
f = open('outgan_train_c100256.txt', 'a+')

# Changing standard out to file out. 
sys.stdout = f
# This will write to the file. 
print("xyz") 

In [ ]:
# %%time
advGAN.train(dataloader, 600)

In [ ]:
# Closing the file.
f.close()

# replacing the original output format to stdout.
sys.stdout = orig_stdout

# This will print onto the console.
print("xyz") 

## Testing Attack

In [ ]:
#Adversarial NMI, ARI, F, and ACC
device = 'cuda'
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = './models/netG_cc_cifar100_epoch_570.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import pyplot
import numpy as np
def plot_examples(images, labels):
    print(type(images[0]), type(labels))
    print(images.shape)
    w = 10
    h = 10
    fig = plt.figure(figsize=(10, 20))
    columns = 11
    rows = 12
    for i in range(10):
#         img = np.random.randint(10, size=(h,w))
        fig.add_subplot(rows, columns, i+1)
#         img = images[i] / 2 + 0.5   # unnormalize
        img = images[i]
        npimg = img.detach().cpu().numpy()   # convert from tensor
        
#         plt.imshow(np.transpose(npimg, (1, 2, 0))) 
#         plt.imshow(npimg)
#   plt.show()
        plt.imshow(np.transpose(np.reshape(images[i].detach().cpu(), (3, 32,32)), (1,2,0)))
        plt.title('#{}: {}'.format(i, labels[i]))
    plt.show()

In [ ]:
# 'mean': [0.5071, 0.4867, 0.4408], 'std': [0.2675, 0.2565, 0.2761]


def save_examples(images, labels, noise=False, bno=0, adv=False, orig=False):
    print(type(images[0]), type(labels))
#     MEAN = torch.tensor([0.4914, 0.4822, 0.4465]).cuda() #c10
#     STD = torch.tensor([0.2023, 0.1994, 0.2010]).cuda() #c10
#     MEAN = torch.tensor([0.5071, 0.4867, 0.4408]).cuda()
#     STD = torch.tensor([0.2675, 0.2565, 0.2761]).cuda()
    MEAN = torch.tensor([0.5071, 0.4867, 0.4408]).cuda() #c100
    STD = torch.tensor([0.2675, 0.2565, 0.2761]).cuda() #c100
    
    for i in range(min(len(images), 20)):
        img = images[i]
        img = img * STD[:, None, None] + MEAN[:, None, None]
        npimg = img.detach().cpu().numpy()   # convert from tensor
        npimg = np.clip(npimg, 0, 1)
        if orig:
#             npimg = np.transpose(npimg, (1, 2, 0))
            plt.imsave(f'../Images/C100/NNM/orig/NNM_c100_b{bno}_{i}_lab{labels[i]:02}.png', npimg.T, dpi=600)
            continue
        if adv:
#             npimg = np.transpose(npimg, (1, 2, 0))
            plt.imsave(f'../Images/C100/NNM/adv/NNM_c100_b{bno}_{i}_lab{labels[i]:02}.png', npimg.T, dpi=600)
            continue
        if noise:
            npimg = npimg / 2 + 0.5 
            plt.imsave(f'../Images/C100/NNM/noise/NNM_c100_b{bno}_{i}_noise_lab{labels[i]:02}.png', npimg.T, dpi=600)
            continue

In [ ]:
@torch.no_grad()
def get_predictions_adv(p, dataloader, model, return_features=False):
    # Make predictions on a dataset with neighbors
    model.eval()
    predictions = [[] for _ in range(p['num_heads'])]
    probs = [[] for _ in range(p['num_heads'])]
    targets = []
    if return_features:
        ft_dim = get_feature_dimensions_backbone(p)
        features = torch.zeros((len(dataloader.sampler), ft_dim)).cuda()
    
    if isinstance(dataloader.dataset, NeighborsDataset): # Also return the neighbors
        key_ = 'anchor'
        include_neighbors = True
        neighbors = []

    else:
        key_ = 'image'
        include_neighbors = False

    ptr = 0
    fla = True
    for step, batch in enumerate(dataloader):
        images = batch['image']
        labels = batch['target']
        images = images.cuda(non_blocking=True)
        labels = labels.cuda(non_blocking=True)
        bs = images.shape[0]
        perturbation = pretrained_G(images)
        perturbation = torch.clamp(perturbation, -0.15, 0.15)
        adv_imgs = perturbation + images
#         adv_imgs = torch.clamp(adv_imgs, 0, 1)
        
        if fla:
            fla = False
            plot_examples(images, [0]*len(adv_imgs))
            plot_examples(adv_imgs, [0]*len(adv_imgs))
        
        res = model(adv_imgs, forward_pass='return_all')
        output = res['output']

        if return_features:
            features[ptr: ptr+bs] = res['features']
            ptr += bs
        for i, output_i in enumerate(output):
            predictions[i].append(torch.argmax(output_i, dim=1))
            probs[i].append(F.softmax(output_i, dim=1))
        targets.append(labels)
        if include_neighbors:
            neighbors.append(batch['possible_neighbors'])

    predictions = [torch.cat(pred_, dim = 0).cpu() for pred_ in predictions]
    probs = [torch.cat(prob_, dim=0).cpu() for prob_ in probs]
    targets = torch.cat(targets, dim=0)

    if include_neighbors:
        neighbors = torch.cat(neighbors, dim=0)
        out = [{'predictions': pred_, 'probabilities': prob_, 'targets': targets, 'neighbors': neighbors} for pred_, prob_ in zip(predictions, probs)]

    else:
        out = [{'predictions': pred_, 'probabilities': prob_, 'targets': targets} for pred_, prob_ in zip(predictions, probs)]

    if return_features:
        return out, features
    else:
        return out

In [ ]:
@torch.no_grad()
def get_predictions_adv_save(p, dataloader, model, return_features=False):
    # Make predictions on a dataset with neighbors
    model.eval()
    predictions = [[] for _ in range(p['num_heads'])]
    probs = [[] for _ in range(p['num_heads'])]
    targets = []
    if return_features:
        ft_dim = get_feature_dimensions_backbone(p)
        features = torch.zeros((len(dataloader.sampler), ft_dim)).cuda()
    
    if isinstance(dataloader.dataset, NeighborsDataset): # Also return the neighbors
        key_ = 'anchor'
        include_neighbors = True
        neighbors = []

    else:
        key_ = 'image'
        include_neighbors = False

    ptr = 0
    fla = True
    for step, batch in enumerate(dataloader):
        images = batch['image']
        labels = batch['target']
        images = images.cuda(non_blocking=True)
        labels = labels.cuda(non_blocking=True)
        bs = images.shape[0]
        perturbation = pretrained_G(images)
        perturbation = torch.clamp(perturbation, -0.15, 0.15)
        adv_imgs = perturbation + images
#         adv_imgs = torch.clamp(adv_imgs, 0, 1)
        
        if fla:
            fla = False
            plot_examples(images, [0]*len(adv_imgs))
            plot_examples(adv_imgs, [0]*len(adv_imgs))
        
        res = model(adv_imgs, forward_pass='return_all')
        output = res['output']

        if return_features:
            features[ptr: ptr+bs] = res['features']
            ptr += bs
        for i, output_i in enumerate(output):
            predictions[i].append(torch.argmax(output_i, dim=1))
            probs[i].append(F.softmax(output_i, dim=1))
        targets.append(labels)
        save_examples(images, labels.data.cpu().numpy(), bno=step, orig=True)
        save_examples(adv_imgs, predictions[-1][0].cpu().numpy(), bno=step, adv=True)
        save_examples(perturbation, labels.data.cpu().numpy(), bno=step, noise=True)
        if include_neighbors:
            neighbors.append(batch['possible_neighbors'])

    predictions = [torch.cat(pred_, dim = 0).cpu() for pred_ in predictions]
    probs = [torch.cat(prob_, dim=0).cpu() for prob_ in probs]
    targets = torch.cat(targets, dim=0)

    if include_neighbors:
        neighbors = torch.cat(neighbors, dim=0)
        out = [{'predictions': pred_, 'probabilities': prob_, 'targets': targets, 'neighbors': neighbors} for pred_, prob_ in zip(predictions, probs)]

    else:
        out = [{'predictions': pred_, 'probabilities': prob_, 'targets': targets} for pred_, prob_ in zip(predictions, probs)]

    if return_features:
        return out, features
    else:
        return out

In [ ]:
#saving samples
# predictions, features = get_predictions_adv_save(config, dataloader, model, return_features=True)

In [ ]:
if config['setup'] in ['scan', 'selflabel']:
        print(colored('Perform evaluation of the clustering model (setup={}).'.format(config['setup']), 'blue'))
        head = state_dict['head'] if config['setup'] == 'scan' else 0
        predictions, features = get_predictions_adv(config, dataloader, model, return_features=True)
        clustering_stats_adv = hungarian_evaluate(head, predictions, dataset.classes, 
                                                compute_confusion_matrix=True, 
                            confusion_matrix_file=None)
        print(clustering_stats)
        print(clustering_stats_adv)

# Transferability

In [ ]:
#Adversarial NMI, ARI, F, and ACC
device = 'cuda'
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = '../Generator_Models/CIFAR100/netG_cc_CIFAR-100_epoch_120.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

if config['setup'] in ['scan', 'selflabel']:
        print(colored('Perform evaluation of the clustering model (setup={}).'.format(config['setup']), 'blue'))
        head = state_dict['head'] if config['setup'] == 'scan' else 0
        predictions, features = get_predictions_adv(config, dataloader, model, return_features=True)
        clustering_stats_adv = hungarian_evaluate(head, predictions, dataset.classes, 
                                                compute_confusion_matrix=True, 
                            confusion_matrix_file=None)
        print(clustering_stats)
        print(clustering_stats_adv)

In [ ]:
#Adversarial NMI, ARI, F, and ACC
device = 'cuda'
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = '../Generator_Models/CIFAR100/netG_SCAN_CIFAR20.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

if config['setup'] in ['scan', 'selflabel']:
        print(colored('Perform evaluation of the clustering model (setup={}).'.format(config['setup']), 'blue'))
        head = state_dict['head'] if config['setup'] == 'scan' else 0
        predictions, features = get_predictions_adv(config, dataloader, model, return_features=True)
        clustering_stats_adv = hungarian_evaluate(head, predictions, dataset.classes, 
                                                compute_confusion_matrix=True, 
                            confusion_matrix_file=None)
        print(clustering_stats)
        print(clustering_stats_adv)

In [ ]:
#Adversarial NMI, ARI, F, and ACC
device = 'cuda'
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = '../Generator_Models/CIFAR100/netG_RUC_CIFAR20.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

if config['setup'] in ['scan', 'selflabel']:
        print(colored('Perform evaluation of the clustering model (setup={}).'.format(config['setup']), 'blue'))
        head = state_dict['head'] if config['setup'] == 'scan' else 0
        predictions, features = get_predictions_adv(config, dataloader, model, return_features=True)
        clustering_stats_adv = hungarian_evaluate(head, predictions, dataset.classes, 
                                                compute_confusion_matrix=True, 
                            confusion_matrix_file=None)
        print(clustering_stats)
        print(clustering_stats_adv)

In [ ]:
#Adversarial NMI, ARI, F, and ACC
device = 'cuda'
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = '../Generator_Models/CIFAR100/netG_SPICE_CIFAR20.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

if config['setup'] in ['scan', 'selflabel']:
        print(colored('Perform evaluation of the clustering model (setup={}).'.format(config['setup']), 'blue'))
        head = state_dict['head'] if config['setup'] == 'scan' else 0
        predictions, features = get_predictions_adv(config, dataloader, model, return_features=True)
        clustering_stats_adv = hungarian_evaluate(head, predictions, dataset.classes, 
                                                compute_confusion_matrix=True, 
                            confusion_matrix_file=None)
        print(clustering_stats)
        print(clustering_stats_adv)

In [ ]:
#Adversarial NMI, ARI, F, and ACC
device = 'cuda'
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = '../Generator_Models/CIFAR100/netG_MICE_CIFAR20.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

if config['setup'] in ['scan', 'selflabel']:
        print(colored('Perform evaluation of the clustering model (setup={}).'.format(config['setup']), 'blue'))
        head = state_dict['head'] if config['setup'] == 'scan' else 0
        predictions, features = get_predictions_adv(config, dataloader, model, return_features=True)
        clustering_stats_adv = hungarian_evaluate(head, predictions, dataset.classes, 
                                                compute_confusion_matrix=True, 
                            confusion_matrix_file=None)
        print(clustering_stats)
        print(clustering_stats_adv)

# Additional Experiemnts

In [ ]:
@torch.no_grad()
def get_predictions_adv_norm(p, dataloader, model, return_features=False, clamping=0.1):
    # Make predictions on a dataset with neighbors
    model.eval()
    predictions = [[] for _ in range(p['num_heads'])]
    probs = [[] for _ in range(p['num_heads'])]
    targets = []
    if return_features:
        ft_dim = get_feature_dimensions_backbone(p)
        features = torch.zeros((len(dataloader.sampler), ft_dim)).cuda()
    
    if isinstance(dataloader.dataset, NeighborsDataset): # Also return the neighbors
        key_ = 'anchor'
        include_neighbors = True
        neighbors = []

    else:
        key_ = 'image'
        include_neighbors = False

    ptr = 0
    fla = True
    norm = 0.0
    for step, batch in enumerate(dataloader):
        images = batch['image']
        labels = batch['target']
        images = images.cuda(non_blocking=True)
        labels = labels.cuda(non_blocking=True)
        bs = images.shape[0]
        perturbation = pretrained_G(images)
        perturbation = torch.clamp(perturbation, -clamping, clamping)
        norm += torch.mean(torch.norm(perturbation.view(perturbation.shape[0], -1), 2, dim=1)).to('cpu').item()
        adv_imgs = perturbation + images
#         adv_imgs = torch.clamp(adv_imgs, 0, 1)
        
        res = model(adv_imgs, forward_pass='return_all')
        output = res['output']

        if return_features:
            features[ptr: ptr+bs] = res['features']
            ptr += bs
        for i, output_i in enumerate(output):
            predictions[i].append(torch.argmax(output_i, dim=1))
            probs[i].append(F.softmax(output_i, dim=1))
        targets.append(labels)
        if include_neighbors:
            neighbors.append(batch['possible_neighbors'])

    predictions = [torch.cat(pred_, dim = 0).cpu() for pred_ in predictions]
    probs = [torch.cat(prob_, dim=0).cpu() for prob_ in probs]
    targets = torch.cat(targets, dim=0)

    if include_neighbors:
        neighbors = torch.cat(neighbors, dim=0)
        out = [{'predictions': pred_, 'probabilities': prob_, 'targets': targets, 'neighbors': neighbors} for pred_, prob_ in zip(predictions, probs)]

    else:
        out = [{'predictions': pred_, 'probabilities': prob_, 'targets': targets} for pred_, prob_ in zip(predictions, probs)]

    if return_features:
        return out, features, norm/len(dataloader)
    else:
        return out

In [ ]:
#Adversarial NMI, ARI, F, and ACC
device = 'cuda'
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = './models/netG_cc_cifar100_epoch_600.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

In [ ]:
norm_l = []
nmi_l = []
ari_l = []
acc_l = []
# clamp = [j for j in range(0, 1, 0.02)]
# clamp = [j for j in np.arange(0, 1.05, 0.05)]
# clamp = [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
clamp = [0, 0.001, 0.003, 0.005, 0.007, 0.01, 0.015, 0.02, 0.025, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
print(clamp)

for j in clamp:
    torch.cuda.empty_cache()
#     acc2, mice_pi_acc, nmi, ari, norm = get_MiCE_adv_performance_norm(model, model_ema, elbo, full_loader, n_data, n_class, clamping=j)
    predictions, features, norm = get_predictions_adv_norm(config, dataloader, model, return_features=True, clamping=j)
    head = state_dict['head'] if config['setup'] == 'scan' else 0
    
    clustering_stats_adv = hungarian_evaluate(head, predictions, dataset.classes, 
                                            compute_confusion_matrix=True, 
                        confusion_matrix_file=None)
#         print(clustering_stats)
    nmi = clustering_stats_adv['NMI']
    ari = clustering_stats_adv['ARI']
    acc = clustering_stats_adv['ACC']
    
    print(f'clamp {j} avg norm: {norm}')
    print('NMI = {:.4f} ARI = {:.4f} ACC = {:.4f}'.format(nmi, ari, acc))
    norm_l.append(norm)
    nmi_l.append(nmi)
    ari_l.append(ari)
    acc_l.append(acc)

In [ ]:
# plt.plot(x, y, label = "line 1", linestyle="-")
# plt.plot(y, x, label = "line 2", linestyle="--")
# plt.plot(x, np.sin(x), label = "curve 1", linestyle="-.")
# plt.plot(x, np.cos(x), label = "curve 2", linestyle=":")
import matplotlib.pyplot as plt
from matplotlib import pyplot

plt.plot(norm_l, nmi_l, label = "nmi", linestyle="-")
plt.plot(norm_l, ari_l, label = "ari", linestyle="-")
plt.plot(norm_l, acc_l, label = "acc", linestyle="-")
plt.xlabel("Perturbation Norm")
plt.ylabel("Performace")
plt.legend()
plt.grid()
plt.savefig('nnm_cifar10.png')
plt.show()

In [ ]:
print(norm_l)
print()
print(nmi_l)
print()
print(ari_l)
print()
print(acc_l)

In [ ]:
import eval_cus

norm_l = []
nmi_l = []
ari_l = []
acc_l = []
# clamp = [j for j in range(0, 1, 0.02)]
# clamp = [j for j in np.arange(0, 1.05, 0.05)]
# clamp = [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
# clamp = [0, 0.001, 0.003, 0.005, 0.007, 0.01, 0.015, 0.02, 0.025, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
clamp = [0, 0.1, 1]
print(clamp)

for j in clamp:
    torch.cuda.empty_cache()
#     acc2, mice_pi_acc, nmi, ari, norm = get_MiCE_adv_performance_norm(model, model_ema, elbo, full_loader, n_data, n_class, clamping=j)
    predictions, features, norm = get_predictions_adv_norm(config, dataloader, model, return_features=True, clamping=j)

    out = predictions[0]['predictions']
    targets = predictions[0]['targets']
    clustering_stats_adv = eval_cus.check(targets, out, 20, dataset.classes, 
                                            compute_confusion_matrix=True, 
                        confusion_matrix_file=None, cf20=True, output_file2=f'NNM_c100_{j}_n{norm}.pdf')
    #         print(clustering_stats)
    nmi = clustering_stats_adv['NMI']
    ari = clustering_stats_adv['ARI']
    acc = clustering_stats_adv['ACC']
    
    print(f'clamp {j} avg norm: {norm}')
    print('NMI = {:.4f} ARI = {:.4f} ACC = {:.4f}'.format(nmi, ari, acc))
    norm_l.append(norm)
    nmi_l.append(nmi)
    ari_l.append(ari)
    acc_l.append(acc)

In [ ]:
predictions, features, norm = get_predictions_adv_norm(config, dataloader, model, return_features=True, clamping=0.1)

out = predictions[0]['predictions']
targets = predictions[0]['targets']
clustering_stats_adv = eval_cus.check(targets, out, 20, dataset.classes, 
                                        compute_confusion_matrix=True, 
                    confusion_matrix_file=None, cf20=True)

In [ ]:
print(clustering_stats_adv)

In [ ]:
print(norm_l)
print()
print(nmi_l)
print()
print(ari_l)
print()
print(acc_l)